# Project: Search and Sample Return


## [Rubric Points](https://review.udacity.com/#!/rubrics/916/view)
---

## Project folder structure

**Test Dataset Path** : ./test_dataset 
**Calibration Images** : ./new_calib

# Notebook Analysis

[//]: # (Image References)
[image1]: ./writeup_images/threshold_tool.png
[image2]: ./calibration_images/example_grid1.jpg
[image3]: ./calibration_images/example_rock1.jpg 


## Color Thresholding : 
For color thresholding, the color_thresh() method was modified to use both a lower bound and an upper bound threshold.
This code is present in : ./code/perception.py (and also in the jupyter notebook).

```python
def color_thresh(img, rgb_thresh_min, rgb_thresh_max):
    color_select = np.zeros_like(img[:,:,0])
    # Threshold with upper and lower bound
    mask = (img[:,:,0] >= rgb_thresh_min[0]) & (img[:,:,1] >= rgb_thresh_min[1]) & (img[:,:,2] >= rgb_thresh_min[2]) \
       & (img[:,:,0] <= rgb_thresh_max[0]) & (img[:,:,1] <= rgb_thresh_max[1]) & (img[:,:,2] <= rgb_thresh_max[2])
    color_select[mask] = 1
    return color_select
```

However, a major problem was setting the actual values for the different variations in obstacle, rock, shadows, etc. I tried the following two approaches :

**1. Using HSV color space using OpenCV2. **
```python
hsv_image = cv2.cvtColor(test_image, cv2.COLOR_RGB2HSV_FULL)
```
However, this didn't make much of a difference and I still faced the same issues differentiating some tricky parts (e.g. the rock and the skid marks of the wheels). So I went back to using the RGB color space.

**2. Creating a dedicated RGB thresholding tool using GUI sliders in python**

This greatly helped me fine tune the threshold and achieve a high degree of accuracy. This tool was written in a Jupyter notebook and the path is : ./code/Thresholding_Tool.ipynb

Screenshot of this tool (note the GUI sliders at the bottom) : 
![Thresholding tool created using python][image1]


## Obstacle detection : 

For obstacle detection, first of all, the thresholds were set using the above tool that I wrote. After that, the following method was used : 

```python
def obstacle_thresh(img, clip_top_ratio = 0.45, rgb_thresh_min=(0, 0, 0), rgb_thresh_max=(118,103,120)):
    obs_th_img = color_thresh(img, rgb_thresh_min, rgb_thresh_max)
    top_half = int(clip_top_ratio * img.shape[0])
    obs_th_img[:top_half,:] = 1 # above the horizon its always an obstacle
    return obs_th_img
```

This method is defined in ./code/perception.py

Firstly, a basic threshold is done on the image to identify the 'walls' / 'hills'.

Secondly, a key approximation has been used : Since the terrain is relatively flat, the sky covers the top part of the image. This sky is added to the 'obstacle' part of the image. In the above example, the top 45% of the image is considered an obstacle (its above the horizon). The rest part of the image that is within the threshold defined, also forms the obstacle.


## Rock samples detection :

For rock sample detection, the rock_thresh() method has been used and is defined in : ./code/perception.py

```python
def rock_thresh(img, rgb_thresh_min=(125,102,0), rgb_thresh_max=(204,185,78)):
    output = color_thresh(img, rgb_thresh_min, rgb_thresh_max)
    output = cv2.GaussianBlur(output, (3, 3), 0)
    return output
```
After thresholding the image, a gaussian blur has been applied. This (the gaussian blur) reduced noise and prevented stray pixels near the rover skid tracks from being detected as a rock.

## Navigable terrain detection : 

For navigable terrain detection, the approach is similar to the one described in the obstacle detection section.

This has been defined in navigable_thresh() method in ./code/perception.py


## Perspective Transform :

A key point of difference between what was shown in the tutorial and what I've done is that - I've applied the perspective transform *after* doing the color threshold. Generally speaking, this shouldn't make a difference but for my approach of thresholding (clipping above/below the horizon, etc) this is an important step.

The perspective transform is defined using the method : perspect_transform() in ./code/perception.py

## The process_image() function :
This function is present in the jupyter notebook : ./code/Rover_Project_Test_Notebook.ipynb


## World map creation :



## Videos :
Video with the given data-set : 
***./output/test_mapping.mp4***

Video with the recorded data-set : 
***./output/test_mapping_recorded.mp4***

### Autonomous Navigation and Mapping

#### 1. Fill in the `perception_step()` (at the bottom of the `perception.py` script) and `decision_step()` (in `decision.py`) functions in the autonomous mapping scripts and an explanation is provided in the writeup of how and why these functions were modified as they were.


#### 2. Launching in autonomous mode your rover can navigate and map autonomously.  Explain your results and how you might improve them in your writeup.  

**Note: running the simulator with different choices of resolution and graphics quality may produce different results, particularly on different machines!  Make a note of your simulator settings (resolution and graphics quality set on launch) and frames per second (FPS output to terminal by `drive_rover.py`) in your writeup when you submit the project so your reviewer can reproduce your results.**

Here I'll talk about the approach I took, what techniques I used, what worked and why, where the pipeline might fail and how I might improve it if I were going to pursue this project further.  